<h1 align=center face="verdana" style="color:#6699FF";font-family:verdana>Utilization of Machine Learning to Predict Titanic Passenger Fate</h1>
<h3 align=center>Meera Lakhavani</h3>
<h4 align=center>9/25/2015</h4>

This Python code follows the data science workflow and implements **logistic regression**, **random forests**, and **support vector machines** from the scikit-learn package to predict survival of passengers onboard the Titanic.

<h2 style="color:#6699FF">1) Read Data</h2>

We are given a full set of data (<font face="courier">titanic_full.csv</font>) with information about Titanic passengers and their survival outcome. I utilized pandas to read the .csv file because it allows me to import the file easily and clean it effectively.

In [ ]:
import pandas as pd
titanic_full = pd.read_csv("titanic_full.csv")

I chose to print the first few rows of <font face="courier">titanic_full</font> to insure that it was read properly. This also allows me to glance at possible gaps in the data.

In [ ]:
titanic_full.head(5)

<h2 style="color:#6699FF"> 2) Explore Data</h2>

Here, I begin an exploration looking for gaps in the data. Specifically I will search for missing and NaN values. I will also start making hypotheses on which factors could influence survival. I will consider the dismissal of variables that are unlikely to have an impact.

In [ ]:
# Search for unique values in categorical variable: embarked
# Use this as a tool to seek NaN values
print(titanic_full["embarked"].unique())

A 'nan' value is initially seen when searching for unique categorical variables in the "embarked" column. In order to have data that can be processed by scikit-learn algorithms, we must fill in these missing values.

One approach is to replace missing values with the most commonly occuring value. To find this value, I will use pandas <font face="courier">.value_counts()</font>.

In [ ]:
embarked_counts = titanic_full["embarked"].value_counts()
embarked_counts

The <font face="courier">.value_counts()</font> command shows us that the vast majority of embarkments were from 'S'. With some confidence, we can fill in NaN values with 'S'.

A quick way to continue our exploration is to use <font face="courier">.isnull().any()</font> to return columns that contain missing values.

In [ ]:
# Return booleans regarding whether the column contains null values
titanic_full.isnull().any()

Our takeaways from this exploration have given us more clarity on what needs to be cleaned. Specifically, values must be filled in for "embarked", "fare", and "age".

<h2 style="color:#6699FF"> 3) Clean Data</h2>

My approach for cleaning the data is as follows:

**1. Fill in missing values**
* Replace missing age values with median age
* Replace missing fare values with median fares
* Replace missing categorical values from embarked column with most common occurence (S)

**2. Ensure all values are numeric** so that scikit-learn can process and train on the data frame
* Binary assignments for gender are suitable (1 and 0)
* Embarked codes can be assigned dummy numeric variables

**3. Use given information to create useful variables**
I decided to create categrorical variables from the name column for titles (Miss., Sir, Captain, etc.) to see if it has an impact on survival.

**4. Determine which columns should be ignored as predictors**
* Remove columns that have obvious and direct impact on survival (survived, body, boat)
* Remove columns with little or no statistical significance (ticket number, cabin, home destination)
    * If desired, we can analyze these factors further later (ex. socioeconomic status of destination, cabin proximity to lifeboats)


<u>This leaves us with **fare**, **embarked**, **age**, **sex**, **sibsp**, **pclass**, **parch**, and **title** as predictor varibles.</u>

In [ ]:
# Replace missing age values with median age
titanic_full["age"] = titanic_full["age"].fillna(titanic_full["age"].median())

#replace missing fares with median
titanic_full["fare"] = titanic_full["fare"].fillna(titanic_full["fare"].median())

In [ ]:
# Replace missing values with code S
titanic_full["embarked"] = titanic_full["embarked"].fillna("S")

# Create dummy variables for embarked column -- three new columns with binary responses to category
## embarked_dummies = pd.get_dummies(titanic_full['embarked'])

# Confirm that the correct columns were created
## print embarked_dummies.head(3)

# Combine data set with new variable columns in new data frame
## titanic_full_new = pd.concat([titanic_full, embarked_dummies], axis=1); titanic_full_new

#Confirm that new data frame is correct
## list(titanic_full_new.columns.values)

# Remove original embarked column now that we have assigned dummy variables
# just kidding dont remove yet because dummies arent working
#titanic_full_new = titanic_full_new.drop('embarked', 1)

# ... until I get my embarked dummy columns to work, we need to convert S, Q, C to numbers

# Replace missing values with code S
#titanic_full.loc["embarked"] = titanic_full["embarked"].fillna("S")

#assign code 0 to S
titanic_full.loc[titanic_full["embarked"] == "S", "embarked"] = 0

#assign code 1 to C
titanic_full.loc[titanic_full["embarked"] == "C", "embarked"] = 1

#assign code 2 to Q
titanic_full.loc[titanic_full["embarked"] == "Q", "embarked"] = 2

# QUESTION FOR JOOLIAN: why when i print this now does it not show in a table like it did before?
print titanic_full.head(3)

In [ ]:
# Replace all the occurences of male with the number 0, and female with 1
titanic_full.loc[titanic_full["sex"] == "male", "sex"] = 0
titanic_full.loc[titanic_full["sex"] == "female", "sex"] = 1

The code below was directly inspired by: https://www.dataquest.io/mission/75/improving-your-submission/

In [ ]:
# ctrl +/ to uncomment the whole thing

# # REVISIT THIS COLUMN CREATION BECAUSE IT IS NOT WORKING RIGHT NOW. DO NOT USE TITLE COLUMNS IN TRAINING RN
# # No we will create a new column for title
# import re

# # A function to get the title from a name
# def get_title(name):
#     # Use a regular expression to search for a title
#     #Titles always consist of capital and lowercase letters, and end with a period
#     title_search = re.search(' ([A-Za-z]+)\.', name)
#     # If the title exists, extract and return it
#     if title_search:
#         return title_search.group(1)
#     return ""

# # Get all the titles and print how often each one occurs
# titles = titanic_full_new['name'].apply(get_title)

# titanic_full_new = pd.concat([titanic_full, titles], axis=1); titanic_full_new

# ##print(pd.value_counts(titles))
# print titles.head(2)
# print titanic_full_new.head(2)
# print titanic_full.head(3)

# ##print titanic_full.head(2)

# ##title_dummies = pd.get_dummies(titanic_full, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False)

# # Create dummy variables for embarked column -- three new columns with binary responses to category
# ##embarked_dummies = pd.get_dummies(titanic_full['title'])

# # Confirm that the correct columns were created
# ##print embarked_dummies.head(5)

# # Combine data set with new variable columns in new data frame
# ##titanic_full_new = pd.concat([titanic_full, embarked_dummies], axis=1); titanic_full_new

# #Confirm that new data frame is correct
# ##list(titanic_full_new.columns.values)

# #DUMMY VARIABLESS

# # Map each title to an integer  Some titles are very rare, and are compressed into the same codes as other titles.
# #title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Dr": 5, "Rev": 6, "Major": 7, "Col": 7, "Mlle": 8, "Mme": 8, "Don": 9, "Lady": 10, "Countess": 10, "Jonkheer": 10, "Sir": 9, "Capt": 7, "Ms": 2, "Dona": 10}
# #for k,v in title_mapping.items():
# #    titles[titles == k] = v

# # Verify that we converted everything.
# #print(pd.value_counts(titles))

# # Add in the title column.
# #titanic_full["title"] = titles

In [ ]:
# Remove unnecessary columns from new data frame
# FIND A CLEANER WAY TO DO THIS
# add name and original embarked column to deletion

for i in ['home.dest', 'body', 'cabin', 'boat', 'ticket']:
    titanic_full.drop(i, 1)

    
###titanic_full_new = titanic_full_new.drop('home.dest', 1)
#titanic_full_new = titanic_full_new.drop('body', 1)
#titanic_full_new = titanic_full_new.drop('cabin', 1)
#titanic_full_new = titanic_full_new.drop('boat', 1)
#titanic_full_new = titanic_full_new.drop('ticket', 1)

print titanic_full.head(2)

In [ ]:
# Check to make sure there are now no null values
titanic_full.isnull().any()

<h2 style="color:#6699FF"> 4) Visualize Data</h2>

Here we can utilize plotting tools (IDENTIFY THEM. MAT PLOT LIB. STATSMODELS? SEABORN? PRETTY STUFF) to continue exploring the data and creating hypotheses on which variables will most influence survival.

In [ ]:
# im thinking.. plots of most likely factors survived vs. not. other types of graphs? hmm
#correlation graphs between certain variable and survival

In [ ]:
# use matplotlib to visualize data with surivival as dependent variable and predictors as independent!! 

In [ ]:
# Import modules
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import seaborn

In [ ]:
# Show plots in the notebook
%matplotlib inline

In [ ]:
# Histogram of sibsp
titanic_full.sibsp.hist()
plt.title('Histogram of Sibsp')
plt.xlabel('Number Siblings/Spouses Onboard')
plt.ylabel('Frequency')

In [ ]:
# barplot of sibsp rating grouped by survival (1 or 2)
pd.crosstab(titanic_full.sibsp, titanic_full.survived.astype(bool)).plot(kind='bar')
plt.title('Sibling/Spouse distrivbution grouped by Survival')
plt.xlabel('Numbers of Sibsp')
plt.ylabel('Frequency')


In [ ]:
# 4D Visualization

# from: https://www.kaggle.com/hekkon/titanic/testing/files

#not working right now but whatever

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for n,point in train.iterrows():
    gender = (1 if point['Male'] else 0 ) + np.random.rand()/10
    pclass = point['Pclass'] + np.random.rand()/10
    surv = point['Survived'] 
    third_feature = point['SibSp'] + np.random.rand()/10
    color = 'red' if surv else 'blue'

    ax.scatter(gender, pclass, third_feature, c=color)

ax.set_xlabel('gender')
ax.set_ylabel('Pclass')
ax.set_zlabel('Sibsp')

plt.show()

plt.savefig('viz.png')

<h2 style="color:#6699FF"> 5) Build Models</h2>

Now, onto the juicy stuff! We will train multiple models based on our (predicted to be) important variables and their effect on survival.

1. Start by splitting our full test set into a training (70%) and test set (30%)
2. Implement **logistic regression**
    SUMMARIZE AND EXPAND ON AND KNOW IMPORTANCE OF LOGISTIC REGRESSION
3. **Random forests** see above for expansion
4. Support vector machines
5. Naive bayes


In [ ]:
# Split full data set for training and testing purposes

from sklearn.cross_validation import train_test_split

titanic_train, titanic_test = train_test_split(titanic_full, test_size = 0.3)

<u><h4 style="color:#6699FF">Logistic Regression</h4></u>

In [ ]:
# Prepare data for logistic regression

y, x = dmatrices('survived ~ sex + age + sibsp + parch + pclass + fare + embarked',
                  titanic_train, return_type="dataframe")

y_test, x_test = dmatrices('survived ~ sex + age + sibsp + parch + pclass + fare + embarked',
                  titanic_test, return_type="dataframe")

print x.columns

In [ ]:
# Flatten y into a 1-D array
y = np.ravel(y)

# Initiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(x, y)

# Check the accuracy on the training set
model.score(x_test, y_test['survived'])

# add .values to end of x and y variables if this stops running

In [ ]:
# Logistic regression model #2 in case we want to alter approach/variables

model2 = LogisticRegression()
model2 = model2.fit(x, y)

# Check the accuracy on the training set
model2.score(x_test, y_test['survived'])

In [ ]:
# Predict class labels for the test set
predicted = model2.predict(x_test)
print predicted

In [ ]:
# Generate class probabilities
#NOTE TO MEERA: TRY TO UNDERSTAND WHAT EACH OF THESE IS DOING STATISTICALLY
probs = model2.predict_proba(x_test)
print probs

In [ ]:
# Generate evaluation metrics
print metrics.accuracy_score(y_test, predicted)
print metrics.roc_auc_score(y_test, probs[:, 1])

In [ ]:
# Confusion matrix
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

In [ ]:
# Evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), x, y, scoring='accuracy', cv=10)
print scores
print scores.mean()

<u><h4 style="color:#6699FF">Random Forests</h4></u>

In [ ]:
from sklearn import cross_validation
from sklearn.ensemble import RandomForestClassifier

predictors = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]

# Confirm that there are no missing values
# is this necessary??
for n in predictors:
    print n
    lista = list(titanic_full_new[n])
    print type(lista)
    print np.isnan(lista).any()

#print titanic_full_new.isnull().any()
#print titanic_full_new.head(5)

In [ ]:
print titanic_full.isnull().any()

In [ ]:
# basic random forests algorithim
# source: https://www.dataquest.io/mission/75/improving-your-submission/

from sklearn import cross_validation
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
#put fare back in
predictors = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]

# n_estimators indicates 100 trees initially

alg = RandomForestClassifier(random_state=1, n_estimators=100, min_samples_split=2, min_samples_leaf=1)

#WHY DOSENT THIS WORK 
scores = cross_validation.cross_val_score(alg, titanic_full[predictors], titanic_full["survived"], cv=3)

#nope below
#scores2 = cross_validation.cross_val_score(alg, titanic_train, titanic_test, cv=3)

print(scores.mean())

In [ ]:
# initializing our rand forest alg function to improve parameters
alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=4, min_samples_leaf=2)

scores = cross_validation.cross_val_score(alg, titanic_full[predictors], titanic_full["survived"], cv=3)

print(scores.mean())

In [ ]:
# selecting the best features

import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif

#implement title into this prediction!!
predictors = ["pclass", "sex", "age", "sibsp", "parch", "fare"]

# Perform feature selection
selector = SelectKBest(f_classif, k=5)
selector.fit(titanic_full[predictors], titanic_full["survived"])

# Get the raw p-values for each feature, and transform from p-values into scores
scores = -np.log10(selector.pvalues_)

# Plot the scores.  See how "Pclass", "Sex", "Title", and "Fare" are the best?
plt.bar(range(len(predictors)), scores)
plt.xticks(range(len(predictors)), predictors, rotation='vertical')
plt.show()

# Pick only the three best features.
predictors = ["pclass", "sex", "title", "fare"]

alg = RandomForestClassifier(random_state=1, n_estimators=150, min_samples_split=8, min_samples_leaf=4)

<u><h4 style="color:#6699FF">Support Vector Machines</h4></u>

In [ ]:
y, x = dmatrices('survived ~ sex + age + sibsp + parch + pclass + fare + embarked',
                  titanic_train, return_type="dataframe")

y_test, x_test = dmatrices('survived ~ sex + age + sibsp + parch + pclass + fare + embarked',
                  titanic_test, return_type="dataframe")

y = np.ravel(y)

from sklearn import svm
clf = svm.LinearSVC()
clf.fit(x,y)

clf2 = svm.SVC()
clf2.fit(x,y)

pred = clf.predict(x_test)

from sklearn.metrics import accuracy_score
acc = accuracy_score(pred, y_test)
print acc

<u><h4 style="color:#6699FF">Naive Bayes</h4></u>

In [ ]:
y, x = dmatrices('survived ~ sex + age + sibsp + parch + pclass + fare + embarked',
                  titanic_train, return_type="dataframe")

y_test, x_test = dmatrices('survived ~ sex + age + sibsp + parch + pclass + fare + embarked',
                  titanic_test, return_type="dataframe")

y = np.ravel(y)

import numpy as np
import pylab as pl

from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(x, y)

pred = clf.predict(x_test)

from sklearn.metrics import accuracy_score
nb_acc = accuracy_score(pred, y_test)
print nb_acc


<h2 style="color:#6699FF"> 6) Validation of Models</h2>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(x_train.values, y_train.values)

In [ ]:
# predict class labels for the test set
predicted = model2.predict(x_test)
print predicted

In [ ]:
# generate class probabilities
probs = model2.predict_proba(x_test)
print probs.head(5)

In [ ]:
# generate evaluation metrics
print metrics.accuracy_score(y_test, predicted)
print metrics.roc_auc_score(y_test, probs[:, 1])

In [ ]:
# confusion matrix
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

In [ ]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), x, y, scoring='accuracy', cv=10)
print scores
print scores.mean()

<font color="#6699FF"> &clubs; &hearts; &diams; &spades; </font>


<h2 style="color:#6699FF"> 7) Appendix</h2>

Sources of code, knowlege, and inspiration:

* Karen Xiao
* Spencer Allee
* Julian Modesto
* Norah (Yuan) Shi

* https://www.udacity.com/course/intro-to-machine-learning--ud120
* https://www.dataquest.io/mission/74/getting-started-with-kaggle/
* http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
* https://www.dataquest.io/mission/75/improving-your-submission/
* https://www.kaggle.com/hekkon/titanic/testing
* https://www.kaggle.com/benhamner/titanic/python-seaborn-pairplot-example
* http://www.myhtmltutorials.com/font.html
* https://www.quora.com/Random-Forests/How-do-random-forests-work-in-laymans-terms
* http://stackoverflow.com/questions/11587782/creating-dummy-variables-in-pandas-for-python
* http://stackoverflow.com/questions/12207326/pandas-frequency-table-for-a-single-variable
* http://stackoverflow.com/questions/11587782/creating-dummy-variables-in-pandas-for-python

